In [61]:
!pip install pydantic_ai
!pip install python-dotenv
!pip install ollama

Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'error'


  error: subprocess-exited-with-error
  
  pip subprocess to install backend dependencies did not run successfully.
  exit code: 1
  
  [31 lines of output]
    Installing build dependencies: started
    Installing build dependencies: finished with status 'done'
    Getting requirements to build wheel: started
    Getting requirements to build wheel: finished with status 'done'
    Preparing metadata (pyproject.toml): started
    Preparing metadata (pyproject.toml): finished with status 'error'
    error: subprocess-exited-with-error
  
    Preparing metadata (pyproject.toml) did not run successfully.
    exit code: 1
  
    [6 lines of output]
    usage: setup.py [global_opts] cmd1 [cmd1_opts] [cmd2 [cmd2_opts] ...]
       or: setup.py --help [cmd1 cmd2 ...]
       or: setup.py --help-commands
       or: setup.py cmd --help
  
    error: invalid command 'dist_info'
    [end of output]
  
    note: This error originates from a subprocess, and is likely not a problem with pip.
  error: 

In [73]:
import nest_asyncio

from pydantic import BaseModel, Field
from pydantic_ai import Agent, RunContext, Tool, ModelRetry
from pydantic_ai.models.openai import OpenAIModel, AsyncOpenAI
from dataclasses import dataclass

import ollama

In [17]:
nest_asyncio.apply()

# USING OLLAMA

In [ ]:
client = AsyncOpenAI(
    base_url='http://localhost:11434/v1',
    api_key='',
)
# Create the agent
ollama_model = OpenAIModel(
    model_name='llama3.2',
    openai_client=client,
)

In [19]:
basic_agent = Agent(
    model=ollama_model,
    system_prompt="You are helpful customer support agent. Be concise and friendly",
)

result = basic_agent.run_sync('How can I track my order #12345?')

In [20]:
print(result.data)

To track your order #12345, please follow these steps:

1. Go to our website and sign in to your account.
2. Click on "Order Tracking" at the top of the page.
3. Enter your order number (#12345) and click "Search".
4. You will be redirected to the shipping carrier's tracking page.

If you're unable to track it online, please contact our customer service team, and we'll be happy to assist you further!


# USING GEMINI

In [21]:
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("GEMINI_API_KEY")

In [22]:
model = 'google-gla:gemini-1.5-flash'

In [13]:
"""
    This example demonstrates the basic usage of PydanticAI agents.
    Key Concepts:
        - Creating a basic agent with a system prompt
        - Running synchronous queries
        - Accessing response data, message history, and costs
"""

'\n    This example demonstrates the basic usage of PydanticAI agents.\n    Key Concepts:\n        - Creating a basic agent with a system prompt\n        - Running synchronous queries\n        - Accessing response data, message history, and costs\n'

In [27]:
basic_agent = Agent(
    model=model,
    system_prompt="You are helpful customer support agent. Be concise and friendly",
)


result = basic_agent.run_sync('How can I track my order #12345?')


In [28]:
print(result.data)


Hi there!  You can track your order #12345 here: [insert tracking link here].  Let me know if you have any other questions!



In [35]:
print(result.all_messages())

[ModelRequest(parts=[SystemPromptPart(content='You are helpful customer support agent. Be concise and friendly', dynamic_ref=None, part_kind='system-prompt'), UserPromptPart(content='How can I track my order #12345?', timestamp=datetime.datetime(2025, 2, 7, 9, 35, 11, 117068, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'), ModelResponse(parts=[TextPart(content='Hi there!  You can track your order #12345 here: [insert tracking link here].  Let me know if you have any other questions!\n', part_kind='text')], model_name='gemini-1.5-flash', timestamp=datetime.datetime(2025, 2, 7, 9, 35, 12, 622610, tzinfo=datetime.timezone.utc), kind='response')]


In [39]:
# print(result.cost())

In [40]:
result2 = basic_agent.run_sync(
    user_prompt="What was my previous question?",
    message_history=result.new_messages()
)

print(result2.data)

Your previous question was how to track your order #12345.



# AGENT WITH STRUCTURED RESPONSE

In [ ]:
"""
    This example shows how to get structured, type-safe responses from the agent.
    Key concepts:
        - Using Pydantic models to define response structure
        - Type validation and safety
        - Field descriptions for better model understanding
"""

In [75]:
# instead of 

"""
    @dataclass
    Model
        - type hints (for ide)
        - data validation
        - serialization (from json | to json)
        - built-in

    Model(BaseModel)
        - type hints (for ide)
        - data validation
        - serialization (from json | to json)
"""


@dataclass
class ResponseModel():
    response: str
    needs_escalation: str
    follow_up_required: str
    sentiment: str = Field(description="Customer sentiment analysis")

In [76]:
# BaseModel -> use Pydantic's validation, serialization from other utilities.
class ResponseModel(BaseModel):
    """ Structured response with metada."""
    response: str
    needs_escalation: str
    follow_up_required: str
    sentiment: str = Field(description="Customer sentiment analysis")

In [44]:
agent1 = Agent(
    model=model,
    result_type=ResponseModel,
    system_prompt=(
        "You are an intelligent customer support agent."
        "Analyze queries carefully and provide structured responses."
    ),
)

response = agent1.run_sync("How can I track my order #12345?")

In [ ]:
print(response.data.model_dump_json(indent=2))

# AGENT WITH STRUCTURED RESPONSE AND DEPENDENCIES

In [104]:
"""
    This example demonstrates how to use dependencies and context in agents.
    Key concepts:
        - Defining complex data models with Pydantic
        - Injectinng runtime dependencies
        - Using dynamic system prompts
"""

'\n    This example demonstrates how to use dependencies and context in agents.\n    Key concepts:\n        - Defining complex data models with Pydantic\n        - Injectinng runtime dependencies\n        - Using dynamic system prompts\n'

In [45]:
from typing import List, Optional

class Order(BaseModel):
    """ Structure for order details """
    order_id: str
    status: str
    items: List[str]

In [46]:
class CustomerDetails(BaseModel):
    """ Structure for incoming customer queries"""

    customer_id: str
    name: str
    email: str
    orders: Optional[List[Order]] = None

In [47]:
# retries -> default = 1

agent3 = Agent(
    model=model,
    result_type=ResponseModel,
    deps_type=CustomerDetails,
    retries=3,
    system_prompt=(
        "You are an intelligent customer support agent. "
        "Analyze queries carefully and provide structured responses. "
        "Always great the customer and provide a helpful response. "
    )
)

In [67]:
def to_markdown(data, indent=0):
    markdown = ""
    
    # Check if the data is an instance of BaseModel and dump the model
    if isinstance(data, BaseModel):
        data = data.model_dump()
    
    # Handle data if it's a dictionary
    if isinstance(data, dict):
        for key, value in data.items():
            markdown += f"{'#' * (indent + 2)} {key.upper()}\n"
            if isinstance(value, (dict, list, BaseModel)):
                markdown += to_markdown(value, indent + 1)
            else:
                markdown += f"{value}\n\n"
    
    # Handle data if it's a list
    elif isinstance(data, list):
        for item in data:
            if isinstance(item, (dict, list, BaseModel)):
                markdown += to_markdown(item, indent + 1)
            else:
                markdown += f"- {item}\n"
        markdown += "\n"
    
    # Handle other types of data (such as strings, integers, etc.)
    else:
        markdown += f"{data}\n\n"
    
    return markdown


In [68]:
# decorator
# Add dynamic system prompt based on dependencies
# CustomerDetails -> dependency
# str -> return type
@agent3.system_prompt
async def add_customer_name(ctx: RunContext[CustomerDetails]) -> str:
    return f"Customer details: {to_markdown(ctx.deps)}"

In [69]:
customer = CustomerDetails(
    customer_id="1",
    name="John Doe",
    email="john.doe@example.com",
    orders=[
        Order(
            order_id="12345",
            status="shipped",
            items = [
                "Blue Jeans",
                "T-Shirt"
            ]
        )
    ]
)

In [70]:
response = agent3.run_sync(user_prompt="What did I order?", deps=customer)

In [71]:
response.all_messages()

[ModelRequest(parts=[SystemPromptPart(content='You are an intelligent customer support agent. Analyze queries carefully and provide structured responses. Always great the customer and provide a helpful response. ', dynamic_ref=None, part_kind='system-prompt'), SystemPromptPart(content='Customer details: ## CUSTOMER_ID\n1\n\n## NAME\nJohn Doe\n\n## EMAIL\njohn.doe@example.com\n\n## ORDERS\n#### ORDER_ID\n12345\n\n#### STATUS\nshipped\n\n#### ITEMS\n- Blue Jeans\n- T-Shirt\n\n\n', dynamic_ref=None, part_kind='system-prompt'), SystemPromptPart(content='Customer details: ## CUSTOMER_ID\n1\n\n## NAME\nJohn Doe\n\n## EMAIL\njohn.doe@example.com\n\n## ORDERS\n#### ORDER_ID\n12345\n\n#### STATUS\nshipped\n\n#### ITEMS\n- Blue Jeans\n- T-Shirt\n\n\n', dynamic_ref=None, part_kind='system-prompt'), SystemPromptPart(content='Customer details: ## CUSTOMER_ID\n1\n\n## NAME\nJohn Doe\n\n## EMAIL\njohn.doe@example.com\n\n## ORDERS\n#### ORDER_ID\n12345\n\n#### STATUS\nshipped\n\n#### ITEMS\n- Blue Jea

In [72]:
print(response.data.model_dump_json(indent=2))

{
  "response": "You ordered a Blue Jeans and a T-Shirt.",
  "needs_escalation": "false",
  "follow_up_required": "false",
  "sentiment": "positive"
}


In [173]:
print(
    "Customer Details:\n"
    f"Name: {customer.name}\n"
    f"Email: {customer.email}\n\n"
    "Response Details:\n"
    f"{response.data.response}\n\n"
    "Status:\n"
    f"Follow-up Required: {response.data.follow_up_required}\n"
    f"Needs Escalation: {response.data.needs_escalation}"
)

Customer Details:
Name: John Doe
Email: john.doe@example.com

Response Details:
You ordered Blue Jeans and a T-Shirt.

Status:
Follow-up Required: false
Needs Escalation: false


# Agent with Tools

In [174]:
""" 
    This example shows how to enhance agents with custom tools.
    Key Concepts:
        - Creating and registering tools
        - Accessing context in tools
"""

' \n    This example shows how to enhance agents with custom tools.\n    Key Concepts:\n        - Creating and registering tools\n        - Accessing context in tools\n'

In [177]:
from typing import Dict

shipping_info_db: Dict[str, str] = {
    "12345": "Shipped on 2024-12-12",
    "67890": "Out for delivery",
}

In [179]:
def get_shipping_info(ctx: RunContext[CustomerDetails]) -> str:
    """ Get the customer's shipping information. """
    return shipping_info_db[ctx.deps.orders[0].order_id]

In [186]:
agent4 = Agent(
    model=model,
    result_type=ResponseModel,
    deps_type=CustomerDetails,
    retries=3,
    system_prompt=(
        "You are an intelligent customer support agent. "
        "Analyze queries carefully and provide structured responses. "
        "Use tools to look up relevant information."
        "Always great the customer and provide a helpful response."
    ),
    tools=[Tool(get_shipping_info, takes_ctx=True)], # Add tool via kwarg
)

In [188]:
@agent4.system_prompt
async def add_customer_name(ctx: RunContext[CustomerDetails]) -> str:
    return f"Customer details: {to_markdown(ctx.deps)}"

In [189]:
response = agent4.run_sync(
    user_prompt="What's the status of my last order?", deps = customer
)

In [191]:
response.all_messages()

[ModelRequest(parts=[SystemPromptPart(content='You are an intelligent customer support agent. Analyze queries carefully and provide structured responses. Use tools to look up relevant information.Always great the customer and provide a helpful response.', dynamic_ref=None, part_kind='system-prompt'), SystemPromptPart(content='Customer details: ## CUSTOMER_ID\n1\n\n## NAME\nJohn Doe\n\n## EMAIL\njohn.doe@example.com\n\n## ORDERS\n#### ORDER_ID\n12345\n\n#### STATUS\nshipped\n\n#### ITEMS\n- Blue Jeans\n- T-Shirt\n\n\n', dynamic_ref=None, part_kind='system-prompt'), UserPromptPart(content="What's the status of my last order?", timestamp=datetime.datetime(2025, 2, 6, 13, 32, 50, 672305, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'),
 ModelResponse(parts=[ToolCallPart(tool_name='get_shipping_info', args={}, tool_call_id=None, part_kind='tool-call')], model_name='gemini-1.5-flash', timestamp=datetime.datetime(2025, 2, 6, 13, 32, 52, 247105, tzinfo=datetime.timezo

In [193]:
print(response.data.model_dump_json(indent=2))

{
  "response": "Your order (12345) has shipped on 2024-12-12.",
  "needs_escalation": "false",
  "follow_up_required": "false",
  "sentiment": "positive"
}


# Agent with Reflection and Self-Correction

In [194]:
"""
    This example demonstrates advanced agent capabilities with self-correction.
    Key concepts:
        - Implementing self-reflection
        - Handling erros gracefully with retries
        - Using ModelRetry for automatic retries
        - Decorator-based tool registration
"""

'\n    This example demonstrates advanced agent capabilities with self-correction.\n    Key concepts:\n        - Implementing self-reflection\n        - Handling erros gracefully with retries\n        - Using ModelRetry for automatic retries\n        - Decorator-based tool registration\n'

In [197]:
shipping_info_db: Dict[str, str] = {
    "#12345": "Shipped on 2024-12-12",
    "#67890": "Out for delivery",
}

In [196]:
customer = CustomerDetails(
    customer_id="1",
    name="John Doe",
    email="john.doe@example.com"
)

In [213]:
agent5 = Agent(
    model=model,
    result_type=ResponseModel,
    deps_type=CustomerDetails,
    retries=3,
    system_prompt=(
        "You are an intelligent customer support agent. "
        "Analyze queries carefully and provide structured responses. "
        "Use tools to look up relevant information. "
        "Always greet the customer and provide a helpful response. "
    )
)

In [214]:
@agent5.tool_plain() # Add plain tool via decorator
def get_shipping_status(order_id: str) -> str:
    """ Get the shipping status for a given order ID."""
    shipping_status = shipping_info_db.get(order_id)
    if shipping_status is None:
        raise ModelRetry(
            f"No shipping information found for order ID {order_id}."
            "Make sure the Order ID starts with a #: e.g. #12345 "
            "Self correct this is needed and try again."
        )
    return shipping_info_db[order_id]

In [215]:
response = agent5.run_sync(
    user_prompt="What's the status of my last order 12345", deps=customer
)

In [216]:
response.all_messages()

[ModelRequest(parts=[SystemPromptPart(content='You are an intelligent customer support agent. Analyze queries carefully and provide structured responses. Use tools to look up relevant information. Always greet the customer and provide a helpful response. ', dynamic_ref=None, part_kind='system-prompt'), UserPromptPart(content="What's the status of my last order 12345", timestamp=datetime.datetime(2025, 2, 6, 13, 45, 18, 517627, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'),
 ModelResponse(parts=[ToolCallPart(tool_name='get_shipping_status', args={'order_id': '12345'}, tool_call_id=None, part_kind='tool-call')], model_name='gemini-1.5-flash', timestamp=datetime.datetime(2025, 2, 6, 13, 45, 20, 114181, tzinfo=datetime.timezone.utc), kind='response'),
 ModelRequest(parts=[RetryPromptPart(content='No shipping information found for order ID 12345.Make sure the Order ID starts with a #: e.g. #12345 Self correct this is needed and try again.', tool_name='get_shippin

In [211]:
print(response.data.model_dump_json(indent=2))

{
  "response": "I am sorry, I cannot find your order information. Please provide a valid order ID.",
  "needs_escalation": "false",
  "follow_up_required": "true",
  "sentiment": "neutral"
}
